In [1]:
import numpy as np
import pandas as pd
import pickle
import calendar
from datetime import datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX

from s3_functions import *

## Functions

In [2]:
def days_in_next_month():
    """Calculate the number of days in the next month.

    This function determines the number of days in the month following
    the current date, accounting for month and year transitions.

    Returns:
        int: Number of days in the next month.
    """
    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month

    next_month = current_month + 1
    next_year = current_year

    if next_month > 12:
        next_month = 1
        next_year += 1

    days_in_next_month = calendar.monthrange(next_year, next_month)[1]

    return days_in_next_month

def sarimax_forecast(riderships_data):
    """Generate a SARIMAX forecast for the next month based on ridership data.

    This function fits a SARIMAX model to the provided ridership data and 
    forecasts values for the number of days in the next month.

    Args:
        riderships_data (pd.DataFrame): A DataFrame containing a column 
            'number_of_riderships' representing the ridership time series data.

    Returns:
        pd.DataFrame: A DataFrame containing the forecasted values for the next month.
            Columns:
            - 'predicted_mean': Forecasted mean values.
            - 'lower_bound': Lower bound of the confidence interval.
            - 'upper_bound': Upper bound of the confidence interval.
    """
    train_series = riderships_data['number_of_riderships']

    sarimax_model = SARIMAX(train_series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
    sarimax_fitted = sarimax_model.fit()

    forecast_steps = days_in_next_month()
    forecast_obj = sarimax_fitted.get_forecast(steps=forecast_steps)
    forecast_mean = forecast_obj.predicted_mean
    conf_int = forecast_obj.conf_int(alpha=0.5)

    forecasts = pd.DataFrame({
        'predicted_mean': forecast_mean,
        'lower_bound': conf_int.iloc[:, 0],
        'upper_bound': conf_int.iloc[:, 1]
    })

    return forecasts

## Import Data

In [4]:
file_key = 'data-transformed/run-1731935541465-part-r-00000'
mta_subway_df = read_s3_csv_to_dataframe(file_key)
mta_subway_df = mta_subway_df.sort_values(by=["station_complex_id", "created_date"], ascending=True)
mta_subway_df.set_index('created_date', inplace=True)
mta_subway_by_station = mta_subway_df[['station_complex_id', 'number_of_riderships']]

In [5]:
mta_subway_df.head()

,station_complex_id,station_complex,latitude,longitude,georeference,number_of_riderships,year_period
created_date,,,,,,,
2023-01-01,8,"5 Av/59 St (N,R,W)",40.764812,-73.97335,POINT (-73.97335 40.764812),8909.0,2023
2023-01-02,8,"5 Av/59 St (N,R,W)",40.764812,-73.97335,POINT (-73.97335 40.764812),7475.0,2023
2023-01-03,8,"5 Av/59 St (N,R,W)",40.764812,-73.97335,POINT (-73.97335 40.764812),9291.0,2023
2023-01-04,8,"5 Av/59 St (N,R,W)",40.764812,-73.97335,POINT (-73.97335 40.764812),10697.0,2023
2023-01-05,8,"5 Av/59 St (N,R,W)",40.764812,-73.97335,POINT (-73.97335 40.764812),10391.0,2023


## Upload Model

In [8]:
with open('sarimax_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

## Make Predictions to all stations

In [9]:
forecasts = mta_subway_by_station.groupby('station_complex_id').apply(sarimax_forecast)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.70735D+00    |proj g|=  7.69288D-02

At iterate    5    f=  8.64986D+00    |proj g|=  1.71960D-02

At iterate   10    f=  8.64591D+00    |proj g|=  2.86929D-03

At iterate   15    f=  8.64554D+00    |proj g|=  2.73729D-04

At iterate   20    f=  8.64531D+00    |proj g|=  5.66779D-03

At iterate   25    f=  8.57845D+00    |proj g|=  6.00440D-02

At iterate   30    f=  8.56154D+00    |proj g|=  6.57655D-03

At iterate   35    f=  8.56119D+00    |proj g|=  7.29406D-04

At iterate   40    f=  8.56112D+00    |proj g|=  1.25972D-04

At iterate   45    f=  8.56111D+00    |proj g|=  1.25745D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = nu

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.



At iterate    5    f=  9.05756D+00    |proj g|=  7.22598D-03

At iterate   10    f=  9.05653D+00    |proj g|=  2.81135D-03

At iterate   15    f=  9.05635D+00    |proj g|=  2.13080D-04

At iterate   20    f=  9.05626D+00    |proj g|=  3.81789D-03

At iterate   25    f=  9.04331D+00    |proj g|=  6.01050D-02

At iterate   30    f=  8.99001D+00    |proj g|=  1.06020D-01

At iterate   35    f=  8.97679D+00    |proj g|=  8.30946D-03

At iterate   40    f=  8.97230D+00    |proj g|=  2.14194D-03

At iterate   45    f=  8.97226D+00    |proj g|=  2.17049D-03

At iterate   50    f=  8.97102D+00    |proj g|=  1.73374D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


...

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.94621D+00    |proj g|=  6.72798D-02

At iterate    5    f=  8.90218D+00    |proj g|=  2.02833D-02

At iterate   10    f=  8.89846D+00    |proj g|=  3.11529D-04

At iterate   15    f=  8.89814D+00    |proj g|=  2.41513D-03

At iterate   20    f=  8.89506D+00    |proj g|=  2.15271D-02

At iterate   25    f=  8.81675D+00    |proj g|=  2.14403D-02

At iterate   30    f=  8.81449D+00    |proj g|=  4.59898D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nac

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.00978D+01    |proj g|=  4.89471D-02

At iterate    5    f=  1.00689D+01    |proj g|=  3.96350D-03

At iterate   10    f=  1.00679D+01    |proj g|=  6.76572D-05

At iterate   15    f=  1.00679D+01    |proj g|=  4.77084D-04

At iterate   20    f=  1.00678D+01    |proj g|=  4.41336D-03

At iterate   25    f=  1.00430D+01    |proj g|=  8.66701D-02

At iterate   30    f=  1.00125D+01    |proj g|=  5.33468D-03

At iterate   35    f=  1.00013D+01    |proj g|=  1.15266D-01

At iterate   40    f=  9.97930D+00    |proj g|=  1.96130D-03

At iterate   45    f=  9.97925D+00    |proj g|=  1.83016D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Ca

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.



At iterate    5    f=  9.10153D+00    |proj g|=  6.15178D-03

At iterate   10    f=  9.09740D+00    |proj g|=  9.69289D-04

At iterate   15    f=  9.09738D+00    |proj g|=  8.51023D-04

At iterate   20    f=  9.09683D+00    |proj g|=  1.26876D-02

At iterate   25    f=  9.01157D+00    |proj g|=  1.56129D-02

At iterate   30    f=  9.00688D+00    |proj g|=  6.77548D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     32     40      1     0     0   3.995D-07   9.007D+00
  F =   9.0068834781163645     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [10]:
forecasts = forecasts.reset_index()
forecasts.columns = ['station_complex_id', 'created_date', 'number_of_riderships', 'lower_bound', 'upper_bound']
forecasts.set_index('created_date', inplace=True)
data_and_predictions = pd.concat([mta_subway_by_station, forecasts], axis=0)

## Save Predictions

In [11]:
save_dataframe_to_s3(data_and_predictions, bucket_name, 'predictions/riderships_predictions.csv')

DataFrame successfully saved to s3://mta-subway/predictions/riderships_predictions.csv
